# I. Set Up

In [1]:
# PYTHON Imports 
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import math
import statistics
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.mplot3d import Axes3D
from pathlib import Path
import glob
import os
import ipywidgets as widgets
from IPython.display import clear_output
import sys
import time
import json
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap
import seaborn as sns
import fnmatch
# ASTROPHY Imports
import astropy 
from astropy.table import Table
from astropy.io import fits
from sherpa.astro import ui
# CIAO Imports
import ciao_contrib.runtool
from ciao_contrib.runtool import *
# CUSTOM Imports
from data_extraction_functions import *
from data_exploration_functions import *
from data_representation_functions import *

# Specify global path
global_path = '/Users/steven/Library/Mobile Documents/com~apple~CloudDocs/0-CfA/4-Data/Datasets'
global_folders = list_folders_fun(global_path)

# Custom object hook to convert lists of lists to NumPy arrays
def numpy_hook(obj):
    if isinstance(obj, list):
        # Check if the list contains sublists (i.e. a matrix)
        if isinstance(obj[0], list):
            # Convert the list of lists to a NumPy array matrix
            return np.array(obj)
    # Return all other objects as is
    return obj

# Select dataset
set_widget = widgets.Dropdown(options=global_folders[:],value=global_folders[0],description='Set :',disabled=False); set_widget

2023-03-24 00:38:01.743666: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Dropdown(description='Set :', options=('All', 'Bona'), value='All')

# II. Load Data

In [3]:
# Set ID
set_id = set_widget.value
# Select Input
files = os.listdir(f'{global_path}/{set_id}/')
input_files = [f for f in files if fnmatch.fnmatch(f, 'hist*nE16*.pkl')]
input_widget = widgets.Dropdown(options=input_files[:],value=input_files[0],description='TSNE File :',disabled=False); input_widget


Dropdown(description='TSNE File :', options=('hist2D-All-nE16-nt24-normnone.pkl', 'hist3D-All-nE16-nt24-ndt24-…

In [4]:
# Load the DataFrame from the CSV file
input_file = input_widget.value
# Load histogram dictionary
with open(f'{global_path}/{set_id}/{input_file}', 'rb') as f:
    hist_dict = pickle.load(f)
# Flatten histograms in the dictionary and get IDs
ids = hist_dict.keys()
histograms = hist_dict.values()
features = np.array([np.array(h).flatten() for h in histograms])
features[np.isnan(features)] = 0.0
# Load properties
df_properties_input = pd.read_csv(f'{global_path}/{set_id}/properties-input-{set_id}.csv')
df_properties_input = df_properties_input[df_properties_input['obsreg_id'].isin(list(ids))]
df_properties = df_properties_input.drop_duplicates('obsreg_id', keep='first').reset_index()

# Print eventfiles and properties number of IDs
print("Number of Features: ", len(features))
print("Number of Property Sets: ", len(df_properties))

Number of Features:  82283
Number of Property Sets:  82283


# OPTIMISATION

In [5]:
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
from scipy.stats import spearmanr
from sklearn.preprocessing import MinMaxScaler

# Select only the columns you need as labels from df_properties
df_label = df_properties[['obsreg_id','hard_hm', 'hard_hs', 'hard_ms', 'var_prob_b', 'var_prob_h', 'var_prob_m', 'var_prob_s']]

mask_nonan = df_label.notna().all(axis=1)
index_nonan = list(df_label.notna().all(axis=1).index[df_label.notna().all(axis=1)])
print(max(index_nonan))
df_label = df_label[mask_nonan]
ID = df_label['obsreg_id'].values
X = features[(index_nonan),:]
# Scale the labels to the same range as the data
scaler = MinMaxScaler()
df_y = df_label[['hard_hm', 'hard_hs', 'hard_ms', 'var_prob_b', 'var_prob_h', 'var_prob_m', 'var_prob_s']]
Y = df_y.values
print(len(X))
print(len(Y))

82273
51830
51830


In [ ]:
pca_model = PCA(100)
X = pca_model.fit_transform(X) 

# identify the elbow point
cumulative_var = np.cumsum(pca_model.explained_variance_ratio_)
elbow_point = np.argmax(np.diff(cumulative_var) <= 0.005) + 1
n_components95 = np.argmax(cumulative_var >= 0.95) + 1
avg = (elbow_point  + n_components95)/2
print('Elbow point:', elbow_point)
print("Number of components to retain 95% variance:", n_components95)
print("Avg:", avg)

# plot the scree plot
ax1 = plt.plot(np.arange(1, pca_model.n_components_ + 1), pca_model.explained_variance_ratio_, 'o-')
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained')
plt.title('Scree Plot')
plt.grid(False) 
plt.axvline(x=elbow_point, color='r')
plt.axvline(x=n_components95, color='b')
plt.axvline(x=avg, color='g')
plt.show()


# plot the cumulative proportion of variance explained
ax2 = plt.plot(np.arange(1, pca_model.n_components_ + 1), np.cumsum(pca_model.explained_variance_ratio_), 'o-')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Proportion of Variance Explained')
plt.title('Scree Plot')
plt.grid(False) 
plt.axvline(x=elbow_point, color='r')
plt.axvline(x=n_components95, color='b')
plt.axvline(x=avg, color='g')
plt.show()




In [6]:
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.neighbors import NearestNeighbors
import random
import dcor
# k = 50
# samples = 5000
# ran = random.randint(samples, 80000)-samples
# X = X[ran:ran+samples,:]
# Y = Y[ran:ran+samples,:]
samples = 51830

pca_model = PCA(21)
PCA = pca_model.fit_transform(X) 
print('PCA DONE')

# define the list of learning rates and perplexities to try
# learning_rates = [50, 100, 250, 500, 750, 1000, 2500, 5000,10000]
learning_rates = [25, 50, 75, 100, 125,150,175,200,225,250]
# perplexities = [25, 50, 100, 250, 500, 750, 1000, 2500,5000]
perplexities = [10,20,30,40,50,60,70,80,90,100]
performance_matrix = np.zeros((len(learning_rates), len(perplexities)))
performance_matrix2 = np.zeros((len(learning_rates), len(perplexities)))
performance_matrix3 = np.zeros((len(learning_rates), len(perplexities)))
performance_matrix4 = np.zeros((len(learning_rates), len(perplexities)))
performance_matrix5 = np.zeros((len(learning_rates), len(perplexities)))
performance_matrix6 = np.zeros((len(learning_rates), len(perplexities)))
rec = []
prec = []

# Compute the inverse covariance matrix
covariance_matrix = np.cov(Y, rowvar=False)
inverse_covariance_matrix = np.linalg.inv(covariance_matrix)
# Compute the Mahalanobis distance matrix for X using pdist
dY2 = pdist(Y, 'mahalanobis', VI=inverse_covariance_matrix)
dY = squareform(dY2)

total_count = len(learning_rates) * len(perplexities)
counter = 0
for i, lr in enumerate(learning_rates):
    for j, perp in enumerate(perplexities):
        # Compute Xtsne embedding
        tsne = TSNE(learning_rate=lr, perplexity=perp, n_iter = 2000, early_exaggeration = 1, init='random')
        X_tsne = tsne.fit_transform(PCA)
        # # Nearest neighbor models
        # nn_X = NearestNeighbors(n_neighbors=k)
        # nn_Y = NearestNeighbors(n_neighbors=k)
        # nn_X.fit(X)
        # nn_Y.fit(Y)
        # _, indices_X = nn_X.kneighbors(X)
        # _, indices_Y = nn_Y.kneighbors(Y)
        # # Compute recall and precision at k
        # recall_at_k = np.mean([np.intersect1d(indices_X[i], indices_Y[i]).shape[0] / k for i in range(samples)])
        # precision_at_k = np.mean([np.intersect1d(indices_X[i], indices_Y[i]).shape[0] / indices_X[i].shape[0] for i in range(samples)])
        # rec.append(recall_at_k)
        # prec.append(precision_at_k)
        # f1 = (2 * precision_at_k * recall_at_k) / (precision_at_k + recall_at_k)
        # performance_matrix3[i,j] = f1
        # compute pairwise distances


        dX2 = pdist(X_tsne,'euclidean')
        dX = squareform(dX2)
        performance_metric = spearmanr(dX.flatten(), dY.flatten())[0]
        performance_metric2 = pearsonr(dX.flatten(), dY.flatten())[0]
        performance_metric3 = spearmanr(dX2.flatten(), dY2.flatten())[0]
        performance_metric4 = pearsonr(dX2.flatten(), dY2.flatten())[0]
        performance_matrix[i,j] = performance_metric 
        performance_matrix2[i,j] = performance_metric2 
        performance_matrix3[i,j] = performance_metric3
        performance_matrix4[i,j] = performance_metric4

        dcor_coefficient1 = dcor.distance_correlation(dX, dY)
        dcor_coefficient2 = dcor.distance_correlation(dX2, dY2)
        performance_matrix5[i,j] = dcor_coefficient1
        performance_matrix6[i,j] = dcor_coefficient2


        counter = counter+1
        clear_output(wait=True)
        print(f'Counter: {counter} of {total_count}')
        print(f'DONE!!!')

# save the matrix to a file
input_file = input_widget.value
data_rep = input_file.split(".pkl")[0]
with open(f'{global_path}/{set_id}/learn-{set_id}-{data_rep}-lr({learning_rates[0]},{learning_rates[-1]})-pp({perplexities[0]},{perplexities[-1]})-sampl{samples}.pkl', 'wb') as f:
    pickle.dump(learning_rates, f)
with open(f'{global_path}/{set_id}/perp-{set_id}-{data_rep}-lr({learning_rates[0]},{learning_rates[-1]})-pp({perplexities[0]},{perplexities[-1]})-sampl{samples}.pkl', 'wb') as f:
    pickle.dump(perplexities, f)
with open(f'{global_path}/{set_id}/perfmatrix-{set_id}-{data_rep}-lr({learning_rates[0]},{learning_rates[-1]})-pp({perplexities[0]},{perplexities[-1]})-sampl{samples}.pkl', 'wb') as f:
    pickle.dump(performance_matrix, f)
with open(f'{global_path}/{set_id}/perfmatrix2-{set_id}-{data_rep}-lr({learning_rates[0]},{learning_rates[-1]})-pp({perplexities[0]},{perplexities[-1]})-sampl{samples}.pkl', 'wb') as f:
    pickle.dump(performance_matrix2, f)
with open(f'{global_path}/{set_id}/perfmatrix3-{set_id}-{data_rep}-lr({learning_rates[0]},{learning_rates[-1]})-pp({perplexities[0]},{perplexities[-1]})-sampl{samples}.pkl', 'wb') as f:
    pickle.dump(performance_matrix3, f)
with open(f'{global_path}/{set_id}/perfmatrix4-{set_id}-{data_rep}-lr({learning_rates[0]},{learning_rates[-1]})-pp({perplexities[0]},{perplexities[-1]})-sampl{samples}.pkl', 'wb') as f:
    pickle.dump(performance_matrix4, f)
with open(f'{global_path}/{set_id}/perfmatrix5-{set_id}-{data_rep}-lr({learning_rates[0]},{learning_rates[-1]})-pp({perplexities[0]},{perplexities[-1]})-sampl{samples}.pkl', 'wb') as f:
    pickle.dump(performance_matrix5, f)
with open(f'{global_path}/{set_id}/perfmatrix6-{set_id}-{data_rep}-lr({learning_rates[0]},{learning_rates[-1]})-pp({perplexities[0]},{perplexities[-1]})-sampl{samples}.pkl', 'wb') as f:
    pickle.dump(performance_matrix5, f)


PCA DONE


: 

: 

In [ ]:
# Select Performance Matrix
files = os.listdir(f'{global_path}/{set_id}/')
perf_files = [f for f in files if fnmatch.fnmatch(f, 'perfmatrix*none*.pkl')]
perf_widget = widgets.Dropdown(options=perf_files[:],value=perf_files[0],description='Performance Matrix :',disabled=False); perf_widget

In [ ]:
# load the matrix from the file
perf_string = perf_widget.value
parts_string = perf_string .split('-')
lr = 'learn-' + '-'.join(parts_string[1:])
perp = 'perp-' + '-'.join(parts_string[1:])
with open(f'{global_path}/{set_id}/{perf_string }', 'rb') as f:
    performance_matrix = pickle.load(f)
with open(f'{global_path}/{set_id}/{lr}', 'rb') as f:
    learning_rates = pickle.load(f)
with open(f'{global_path}/{set_id}/{perp}', 'rb') as f:
    perplexities = pickle.load(f)

fig, ax = plt.subplots()
# Define Font Settings
plt.rcParams.update({'font.size': 10})
plt.rcParams['font.monospace'] = "Courier"
plt.rcParams["font.family"] = "monospace"

im = ax.imshow(performance_matrix, cmap='Spectral')
ax.set_yticks(np.arange(len(learning_rates)),labels = learning_rates)
ax.set_ylabel('learning_rate')
ax.set_xticks(np.arange(len(perplexities)), labels = perplexities)
ax.set_xlabel('perplexity')
ax.grid(False) 

cbar = ax.figure.colorbar(im, ax=ax)
cbar.ax.set_ylabel("performance_metric")
# Loop over data dimensions and create text annotations.
for i in range(len(learning_rates)):
    for j in range(len(perplexities)):
        text = ax.text(j, i, round(performance_matrix[i, j],2),
                       ha="center", va="center", color="w")
        

# III. TSNE

TSNE Model

In [ ]:
# TSNE Setting
n_comp = 2
perp = 1280
learn_rate = 400
early_exag = 1
iterations = 2000

# Define TSNE Model
tsne_model = TSNE(n_components = n_comp , perplexity = perp, learning_rate = learn_rate, n_iter = iterations, early_exaggeration = early_exag, init='random')
# PCA
pca_model = PCA(21)
X_new = pca_model.fit_transform(features) 
# Run TSNE Model on Data
tsne_out = tsne_model.fit_transform(X_new)
print(tsne_out.shape)

# Save TSNE Output
df_tsne = pd.DataFrame(tsne_out, columns=['tsne1', 'tsne2'])
df_tsne['obsreg_id'] = list(ids)
input_file = input_widget.value
data_rep = input_file.split(".pkl")[0]
df_tsne.to_csv(f'{global_path}/{set_id}/tsne-{set_id}-{data_rep}-{n_comp}D-perp{perp}-lr{learn_rate}-ee{early_exag}-it{iterations}.csv', index=False)


Plot TSNE

In [ ]:
files = os.listdir(f'{global_path}/{set_id}/')
tsne_files = [f for f in files if fnmatch.fnmatch(f, 'tsne*nE16*')]
tsne_widget = widgets.Dropdown(options=tsne_files[:],value=tsne_files[0],description='TSNE File :',disabled=False); tsne_widget

In [ ]:
# Load the DataFrame from the CSV file
tsne_file = tsne_widget.value
df_tsne = pd.read_csv(f'{global_path}/{set_id}/{tsne_file}')
# Use loc to select rows that match the IDs in list2, and store the result in a new DataFrame
flares = ['9109_333','9109_344','13637_1078','14368_489','14368_503','14431_16','14542_18','10822_185','10955_21','10996_5','2833_53','13610_112','15214_29']
dips = ['10783_10','10871_10','11059_10','9070_10','9072_10','13814_567','13682_9','1708_192','1708_193','1712_91','15553_237','13681_9','13813_86']
rosanne = ['13814_567']
others = ['10346_11','10542_331','10542_331','10545_496','10556_5752','10556_6687','10811_223','10821_241', '1878_331','10930_1050','10953_275','10956_64']
tsne_flares = df_tsne.loc[df_tsne['obsreg_id'].isin(flares)]
tsne_dips = df_tsne.loc[df_tsne['obsreg_id'].isin(dips)]
tsne_rosanne = df_tsne.loc[df_tsne['obsreg_id'].isin(rosanne)]
tsne_others = df_tsne.loc[df_tsne['obsreg_id'].isin(others)]

In [ ]:
# Plot 2D
fig, axs = plt.subplots(2, 3, figsize=(20, 12),constrained_layout = True)
plt.rcParams.update({'font.size': 10})
plt.rcParams['font.monospace'] = "Courier"
plt.rcParams["font.family"] = "monospace"
fig.suptitle(tsne_file)
colourmap_hard = 'viridis_r'
colourmap_var = 'plasma_r'

hard_hm = axs[0,0].scatter(df_tsne['tsne1'], df_tsne['tsne2'],c=df_properties['hard_hm'], s=0.1, cmap=colourmap_hard)
axs[0,0].set_xlabel('tSNE feature 1')
axs[0,0].set_ylabel('tSNE feature 2')
axs[0,0].set_title(f'hard_hm')
fig.colorbar(hard_hm, ax = axs[0,0])

hard_hs = axs[0,1].scatter(df_tsne['tsne1'], df_tsne['tsne2'], c=df_properties['hard_hs'], s=0.1, cmap=colourmap_hard)
axs[0,1].set_xlabel('tSNE feature 1')
axs[0,1].set_ylabel('tSNE feature 2')
axs[0,1].set_title(f'hard_hs')
fig.colorbar(hard_hs, ax = axs[0,1])

hard_ms = axs[0,2].scatter(df_tsne['tsne1'], df_tsne['tsne2'], c=df_properties['hard_ms'], s=0.1, cmap=colourmap_hard)
axs[0,2].set_xlabel('tSNE feature 1')
axs[0,2].set_ylabel('tSNE feature 2')
axs[0,2].set_title(f'hard_ms')
fig.colorbar(hard_ms, ax = axs[0,2])

var_h = axs[1,0].scatter(df_tsne['tsne1'], df_tsne['tsne2'], c=df_properties['var_prob_h'], s=0.1, cmap=colourmap_var)
axs[1,0].set_xlabel('tSNE feature 1')
axs[1,0].set_ylabel('tSNE feature 2')
axs[1,0].set_title(f'var_prob_h')
fig.colorbar(var_h, ax = axs[1,0])

var_m = axs[1,1].scatter(df_tsne['tsne1'], df_tsne['tsne2'], c=df_properties['var_prob_m'], s=0.1, cmap=colourmap_var)
axs[1,1].set_xlabel('tSNE feature 1')
axs[1,1].set_ylabel('tSNE feature 2')
axs[1,1].set_title(f'var_prob_m')
fig.colorbar(var_m, ax = axs[1,1])

var_s = axs[1,2].scatter(df_tsne['tsne1'], df_tsne['tsne2'], c=df_properties['var_prob_s'], s=0.1, cmap=colourmap_var)
axs[1,2].set_xlabel('tSNE feature 1')
axs[1,2].set_ylabel('tSNE feature 2')
axs[1,2].set_title(f'var_prob_s')
fig.colorbar(var_s, ax = axs[1,2])

for ax in axs.flatten()[0:1]:
    ax.scatter(tsne_flares['tsne1'], tsne_flares['tsne2'], c='red', marker='x', s=100, label='Flares')
    ax.scatter(tsne_dips['tsne1'], tsne_dips['tsne2'], c='blue', marker='x', s=100, label='Dips')
    ax.scatter(tsne_rosanne['tsne1'], tsne_rosanne['tsne2'], c='black', marker='x', s=100, label='Rosanne')
    ax.scatter(tsne_others['tsne1'], tsne_others['tsne2'], c='black', marker='x', s=100, label='Others')
    ax.legend()

plt.show()

# IV. UMAP

UMAP Model

In [ ]:
# pca Setting
n_comp = 2
neighbours = 50
min_d = 0.1
learn_rate = 0.1
sp = 2

# Define pca Model
umap_model = umap.UMAP(n_components = n_comp, n_neighbors = neighbours, min_dist = min_d, learning_rate = learn_rate, random_state=42, spread = sp)
# Run pca Model on Data
umap_out = umap_model.fit_transform(features)
print(umap_out.shape)

# Save pca Output
df_umap = pd.DataFrame(umap_out, columns=['umap1', 'umap2'])
df_umap['obsreg_id'] = list(ids)
input_file = input_widget.value
data_rep = input_file.split(".pkl")[0]
df_umap.to_csv(f'{global_path}/{set_id}/umap-{set_id}-{data_rep}-{n_comp}D-neighb{neighbours}-mind-{min_d}-lr{learn_rate}-sp{sp}.csv', index=False)



Plot UMAP

In [ ]:
files = os.listdir(f'{global_path}/{set_id}/')
umap_files = [f for f in files if fnmatch.fnmatch(f, 'umap*')]
umap_widget = widgets.Dropdown(options=umap_files[:],value=umap_files[0],description='UMAP File :',disabled=False); umap_widget

In [ ]:
# Load the DataFrame from the CSV file
umap_file = umap_widget.value
df_umap = pd.read_csv(f'{global_path}/{set_id}/{umap_file}')
# Use loc to select rows that match the IDs in list2, and store the result in a new DataFrame
flares = ['9109_333','9109_344','13637_1078','14368_489','14368_503','14431_16','14542_18','10822_185','10955_21','10996_5','2833_53','13610_112','15214_29']
dips = ['10783_10','10871_10','11059_10','9070_10','9072_10','13814_567','13682_9','1708_192','1708_193','1712_91','15553_237','13681_9','13813_86']
rosanne = ['13814_567']
umap_flares = df_umap.loc[df_umap['obsreg_id'].isin(flares)]
umap_dips = df_umap.loc[df_umap['obsreg_id'].isin(dips)]
umap_rosanne = df_umap.loc[df_umap['obsreg_id'].isin(rosanne)]

In [ ]:
# Plot
fig, axs = plt.subplots(2, 3, figsize=(20, 12),constrained_layout = True)
fig.suptitle(umap_file)
colourmap_hard = 'viridis_r' #inferno 'viridis_r' 'plasma_r'
colourmap_var = 'plasma_r'

hard_hm = axs[0,0].scatter(df_umap['umap1'], df_umap['umap2'],c=df_properties['hard_hm'], s=0.1, cmap=colourmap_hard)
axs[0,0].set_xlabel('UMAP feature 1')
axs[0,0].set_ylabel('UMAP feature 2')
axs[0,0].set_title(f'hard_hm')
fig.colorbar(hard_hm, ax = axs[0,0])

hard_hs = axs[0,1].scatter(df_umap['umap1'], df_umap['umap2'], c=df_properties['hard_hs'], s=0.1, cmap=colourmap_hard)
axs[0,1].set_xlabel('UMAP feature 1')
axs[0,1].set_ylabel('UMAP feature 2')
axs[0,1].set_title(f'hard_hs')
fig.colorbar(hard_hs, ax = axs[0,1])

hard_ms = axs[0,2].scatter(df_umap['umap1'], df_umap['umap2'], c=df_properties['hard_ms'], s=0.1, cmap=colourmap_hard)
axs[0,2].set_xlabel('UMAP feature 1')
axs[0,2].set_ylabel('UMAP feature 2')
axs[0,2].set_title(f'hard_ms')
fig.colorbar(hard_ms, ax = axs[0,2])

var_h = axs[1,0].scatter(df_umap['umap1'], df_umap['umap2'], c=df_properties['var_prob_h'], s=0.1, cmap=colourmap_var)
axs[1,0].set_xlabel('UMAP feature 1')
axs[1,0].set_ylabel('UMAP feature 2')
axs[1,0].set_title(f'var_prob_h')
fig.colorbar(var_h, ax = axs[1,0])

var_m = axs[1,1].scatter(df_umap['umap1'], df_umap['umap2'], c=df_properties['var_prob_m'], s=0.1, cmap=colourmap_var)
axs[1,1].set_xlabel('UMAP feature 1')
axs[1,1].set_ylabel('UMAP feature 2')
axs[1,1].set_title(f'var_prob_m')
fig.colorbar(var_m, ax = axs[1,1])

var_s = axs[1,2].scatter(df_umap['umap1'], df_umap['umap2'], c=df_properties['var_prob_s'], s=0.1, cmap=colourmap_var)
axs[1,2].set_xlabel('UMAP feature 1')
axs[1,2].set_ylabel('UMAP feature 2')
axs[1,2].set_title(f'var_prob_s')
fig.colorbar(var_s, ax = axs[1,2])

for ax in axs.flatten()[0:1]:
    ax.scatter(umap_flares['umap1'], umap_flares['umap2'], c='red', marker='x', s=100, label='Flares')
    ax.scatter(umap_dips['umap1'], umap_dips['umap2'], c='blue', marker='x', s=100, label='Dips')
    ax.scatter(umap_rosanne['umap1'], umap_rosanne['umap2'], c='black', marker='x', s=100, label='Rosanne')
    ax.legend()

plt.show()

# V. PCA

In [ ]:
# Run PCA
pca_model = PCA(n_components=2)
pca_out = pca_model.fit_transform(features) 

# Save UMAP Output
df_pca = pd.DataFrame(pca_out, columns=['pca1', 'pca2'])
df_pca['obsreg_id'] = list(ids)
input_file = input_widget.value
data_rep = input_file.split(".pkl")[0]
df_pca.to_csv(f'{global_path}/{set_id}/pca-{set_id}-{data_rep}.csv', index=False)

In [ ]:
files = os.listdir(f'{global_path}/{set_id}/')
pca_files = [f for f in files if fnmatch.fnmatch(f, 'pca*')]
pca_widget = widgets.Dropdown(options=pca_files[:],value=pca_files[0],description='PCA File :',disabled=False); pca_widget

In [ ]:
# Load the DataFrame from the CSV file
pca_file = pca_widget.value
df_pca = pd.read_csv(f'{global_path}/{set_id}/{pca_file}')
# Use loc to select rows that match the IDs in list2, and store the result in a new DataFrame
flares = ['9109_333','9109_344','13637_1078','14368_489','14368_503','14431_16','14542_18','10822_185','10955_21','10996_5','2833_53','13610_112','15214_29']
dips = ['10783_10','10871_10','11059_10','9070_10','9072_10','13814_567','13682_9','1708_192','1708_193','1712_91','15553_237','13681_9','13813_86']
rosanne = ['13814_567']
pca_flares = df_pca.loc[df_pca['obsreg_id'].isin(flares)]
pca_dips = df_pca.loc[df_pca['obsreg_id'].isin(dips)]
pca_rosanne = df_pca.loc[df_pca['obsreg_id'].isin(rosanne)]

In [ ]:
# Plot
fig, axs = plt.subplots(2, 3, figsize=(20, 12),constrained_layout = True)
fig.suptitle(pca_file)
colourmap_hard = 'viridis_r' #inferno 'viridis_r' 'plasma_r'
colourmap_var = 'plasma_r'

hard_hm = axs[0,0].scatter(df_pca['pca1'], df_pca['pca2'],c=df_properties['hard_hm'], s=0.1, cmap=colourmap_hard)
axs[0,0].set_xlabel('pca feature 1')
axs[0,0].set_ylabel('pca feature 2')
axs[0,0].set_title(f'hard_hm')
fig.colorbar(hard_hm, ax = axs[0,0])

hard_hs = axs[0,1].scatter(df_pca['pca1'], df_pca['pca2'], c=df_properties['hard_hs'], s=0.1, cmap=colourmap_hard)
axs[0,1].set_xlabel('pca feature 1')
axs[0,1].set_ylabel('pca feature 2')
axs[0,1].set_title(f'hard_hs')
fig.colorbar(hard_hs, ax = axs[0,1])

hard_ms = axs[0,2].scatter(df_pca['pca1'], df_pca['pca2'], c=df_properties['hard_ms'], s=0.1, cmap=colourmap_hard)
axs[0,2].set_xlabel('pca feature 1')
axs[0,2].set_ylabel('pca feature 2')
axs[0,2].set_title(f'hard_ms')
fig.colorbar(hard_ms, ax = axs[0,2])

var_h = axs[1,0].scatter(df_pca['pca1'], df_pca['pca2'], c=df_properties['var_prob_h'], s=0.1, cmap=colourmap_var)
axs[1,0].set_xlabel('pca feature 1')
axs[1,0].set_ylabel('pca feature 2')
axs[1,0].set_title(f'var_prob_h')
fig.colorbar(var_h, ax = axs[1,0])

var_m = axs[1,1].scatter(df_pca['pca1'], df_pca['pca2'], c=df_properties['var_prob_m'], s=0.1, cmap=colourmap_var)
axs[1,1].set_xlabel('pca feature 1')
axs[1,1].set_ylabel('pca feature 2')
axs[1,1].set_title(f'var_prob_m')
fig.colorbar(var_m, ax = axs[1,1])

var_s = axs[1,2].scatter(df_pca['pca1'], df_pca['pca2'], c=df_properties['var_prob_s'], s=0.1, cmap=colourmap_var)
axs[1,2].set_xlabel('pca feature 1')
axs[1,2].set_ylabel('pca feature 2')
axs[1,2].set_title(f'var_prob_s')
fig.colorbar(var_s, ax = axs[1,2])

for ax in axs.flatten():
    ax.scatter(pca_flares['pca1'], pca_flares['pca2'], c='red', marker='x', s=100, label='Flares')
    ax.scatter(pca_dips['pca1'], pca_dips['pca2'], c='blue', marker='x', s=100, label='Dips')
    ax.scatter(pca_rosanne['pca1'], pca_rosanne['pca2'], c='black', marker='x', s=100, label='Rosanne')
    ax.set_xlim([-25,50])
    ax.set_ylim([-50,25])
    ax.legend()

plt.show()

In [ ]:
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr

X = X[0:5000,:]
Y = Y[0:X.shape[0],:]

# define the list of learning rates and perplexities to try
learning_rates = [10, 25, 50, 75, 100, 250, 500]
perplexities = [5, 10, 20, 30, 40, 50,75,100]
performance_matrix = np.zeros((len(learning_rates), len(perplexities)))
total_count = len(learning_rates) * len(perplexities)
counter = 0
for i, lr in enumerate(learning_rates):
    for j, perp in enumerate(perplexities):

        # Compute Xtsne embedding
        tsne = TSNE(learning_rate=lr, perplexity=perp, n_iter = 2000, early_exaggeration = 1, init='random')
        X_tsne = tsne.fit_transform(X)

        # compute pairwise distances
        dY = pdist(Y)
        dY = squareform(dY)
        #dY = scaler.fit_transform(dY)

        # # compute pairwise similarities between labels
        # label_similarities = np.zeros((Y.shape[0], Y.shape[0]))
        # for i in range(Y.shape[0]):
        #     for j in range(i+1, Y.shape[0]):
        #         correlation = np.corrcoef(Y[i,:], Y[j,:])[0,1]
        #         label_similarities[i,j] = correlation
        #         label_similarities[j,i] = correlation

        # # compute the pairwise similarity matrix for the labels JACCARD SIMILARITY
        # label_similarities2 = np.zeros((Y.shape[1], Y.shape[1]))
        # for i in range(Y.shape[0]):
        #     for j in range(i+1, Y.shape[0]):
        #         similarity = np.sum(Y[:,i] & Y[:,j]) / np.sum(Y[:,i] | Y[:,j])
        #         label_similarities[i,j] = similarity
        #         label_similarities[j,i] = similarity

        # compute pairwise distances
        dX = pdist(X_tsne)
        dX = squareform(dX)
        #dX = scaler.fit_transform(dX)

        # # compute pairwise similarities between labels
        # tsne_similarities = np.zeros((X_tsne.shape[0], X_tsne.shape[0]))
        # for i in range(X_tsne.shape[0]):
        #     for j in range(i+1, X_tsne.shape[0]):
        #         distance = np.linalg.norm(X_tsne[i,:]-X_tsne[j,:])
        #         tsne_similarities[i,j] = distance
        #         tsne_similarities[j,i] = distance

        print(dY.shape,dX.shape)
        # print(label_similarities.shape,tsne_similarities.shape)
        performance_metric = spearmanr(dY.flatten(), dX.flatten())[0]
        # performance_metric2 = spearmanr(label_similarities.flatten(), tsne_similarities.flatten())[0]
        performance_matrix[i,j] = performance_metric 
print(performance_matrix)

# save the matrix to a file
input_file = input_widget.value
data_rep = input_file.split(".pkl")[0]
with open(f'{global_path}/{set_id}/perfmatrix-{set_id}-{data_rep}-lr({learning_rates[0]},{learning_rates[-1]})-pp({perplexities[0]},{perplexities[-1]}).pkl', 'wb') as f:
    pickle.dump(performance_matrix, f)
